<a href="https://colab.research.google.com/github/Rajeevku1/AIAgentSamples/blob/main/FlightAssistentMulti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
import gradio as gr
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import userdata
import json
import math


In [2]:
openai_api_key = userdata.get('OPENAI_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

# Initialize OpenAI clients
if openai_api_key:
    openai_client = OpenAI(api_key=openai_api_key)
else:
    openai_client = None


In [3]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""
MODEL = "gpt-4.1-mini"

In [4]:
# Let's start by making a useful function

ticket_prices = {"dallas to london": "$799", "dallas to paris": "$899", "dallas to tokyo": "$1400", "dallas to berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for travel leg {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"


In [5]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
                "start_city": {
                    "type": "string",
                    "description": "The city the customer is departing from"
                },
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to"
                },
                "passenger_name": {
                    "type": "string",
                    "description": "The customer name who wants to travel"
                },
        },
            "required": ["start_city", "destination_city","passenger_name"],
            "additionalProperties": True
    }
}

In [6]:
# And this is included in a list of tools:
tools = [{"type": "function", "function": price_function}]

In [7]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai_client.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [9]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        print(tool_call.function.arguments)
        arguments = json.loads(tool_call.function.arguments)
        startcity = arguments.get('start_city')
        destcity = arguments.get('destination_city')
        passengerName  = arguments.get('passenger_name')
        if not startcity or not destcity:
            return "none"
        print(f"Tool called for {passengerName} travel leg {startcity} to {destcity}")
        travelLeg = startcity + " to " + destcity
        price_details = get_ticket_price(travelLeg)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6908a266c54e25eeda.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ChatCompletion(id='chatcmpl-D0IXQXKhWZnLV3HWqPcBa2VjhdQIz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you with your flight today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768963632, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=13, prompt_tokens=136, total_tokens=149, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletion(id='chatcmpl-D0IXjDiUFaSpoj0KQpGDkxIySMH40', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='May I have your name, please?', refusal=None, role='assistant', annotations=[], 

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1696, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/utils.py", line 882, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6908a266c54e25eeda.gradio.live


In [ ]:
def tavily_search(query):
    url = "https://api.tavily.com/search"
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": query,
        "max_results": 3
    }
    return requests.post(url, json=payload).json()



In [ ]:
def ask_with_internet(question):
    # Step 1: Search
    results = tavily_search(question)

    # Step 2: Extract text
    context = "\n".join([r["content"] for r in results["results"]])

    # Step 3: Ask LLM with context
    prompt = f"""
    Use the following web search results to answer the question.

    Search Results:
    {context}

    Question: {question}

    Answer using the search results.
    """
    #print(prompt)
    return prompt
